In [ ]:
from deap import base
from deap import creator
from deap import tools

import random
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import elitism
import pandas

In [ ]:
#plt.plot(x,function)
#plt.show()

#plt.plot(x,np.power(function,2))
#plt.plot(x,np.power([i*function[i] for i in range(len(function))],2))
#plt.show()

#plt.plot([(function[i]*function[j]*ACF(np.abs(i-j))) for i in range(len(function)) for j in range(len(function))])
#plt.show()

#plt.plot(y,pulse)
#plt.plot()

#plt.plot(np.convolve(pulse, function))
#plt.show()

In [ ]:
"""
x = np.arange(0,80)
a = randomFloat(0,80)
function = -(x-a[0])*(x-a[1])*(x-a[2])*(x-a[3])*(x-a[4])
y = np.arange(0,512)
pulse = [(10 + i*0.1307 + 104*(2*(1-np.power(np.e, (194-i)/27.2))-(1-np.power(np.e, (194-i)/13.6)))) if (i>194) else (10 + i*0.1307) for i in y]

f0 = np.sum(np.power(function,2))
f1 = np.sum(np.power([i*function[i] for i in range(len(function))],2))
f2 = np.sum([(function[i]*function[j]*ACF(np.abs(i-j))) for i in range(len(function)) for j in range(len(function))])
f3 = np.sum(np.convolve(pulse, function) - 104) * np.power(10,5)

print(a)
print(f0)
print(f1)
print(f2)
print(f3)
"""

In [ ]:
#data = pandas.read_csv("001_Mn_1x1mm_fastspectrum_2015-09-17_15-47-raw.csv", delimiter='\t', engine='python', dtype='int', skip_blank_lines=True, usecols= ['CH4'])
#data.info()

In [ ]:
def randomFloat(low, up):
    return [random.uniform(l, u) for l, u in zip([low] * DIMENSIONS, [up] * DIMENSIONS)]

def ACF(x):
    if x == 0:
        return 1
    else:
        return (np.power(0.965,x) - 0.2)

def dpp(individual):
    a0 = individual[0]
    a1 = individual[1]
    a2 = individual[2]
    a3 = individual[3]
    a4 = individual[4]
    
    x = np.arange(0,80)
    function = -(x-a0)*(x-a1)*(x-a2)*(x-a3)*(x-a4)
    
    y = np.arange(0,512)
    pulse = [(10 + i*0.1307 + 104*(2*(1-np.power(np.e, (194-i)/27.2))-(1-np.power(np.e, (194-i)/13.6)))) if (i>194) else (10 + i*0.1307) for i in y]
    
    f0 = np.sum(np.power(function,2))
    f1 = np.sum(np.power([i*function[i] for i in range(len(function))],2))
    f2 = np.sum([(function[i]*function[j]*ACF(np.abs(i-j))) for i in range(len(function)) for j in range(len(function))])
    f3 = np.power(10,5) * np.sum(np.convolve(pulse, function) - 104)
    f = f0+f1+f2+f3
    return -f, 

In [ ]:
DIMENSIONS = 5
BOUND_LOW, BOUND_UP = 0, 80
POPULATION_SIZE = 300
P_CROSSOVER = 0.8 
P_MUTATION = 0.2  
MAX_GENERATIONS = 100
HALL_OF_FAME_SIZE = 30
CROWDING_FACTOR = 20.0 

In [ ]:
toolbox = base.Toolbox()
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox.register("attrFloat", randomFloat, BOUND_LOW, BOUND_UP)
toolbox.register("individualCreator", tools.initIterate, creator.Individual, toolbox.attrFloat)
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)
toolbox.register("evaluate", dpp)
toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=CROWDING_FACTOR)
toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUND_LOW, up=BOUND_UP, eta=CROWDING_FACTOR, indpb=1.0/DIMENSIONS)

In [ ]:
population = toolbox.populationCreator(n=POPULATION_SIZE)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", np.min)
stats.register("avg", np.mean)
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)
population, logbook = elitism.eaSimpleWithElitism(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION, ngen=MAX_GENERATIONS, stats=stats, halloffame=hof, verbose=False)
minFitnessValues, meanFitnessValues = logbook.select("min", "avg")

In [ ]:
best = hof.items[0]
print("-- Best Individual = ", best)
print("-- Best Fitness = ", best.fitness.values[0])

In [ ]:
sns.set_style("whitegrid")
plt.plot(minFitnessValues, color='red')
plt.plot(meanFitnessValues, color='green')
plt.xlabel('Generation')
plt.ylabel('Min / Average Fitness')
plt.title('Min and Average fitness over Generations')

plt.show()